# Model - YOLOv8
# type - One vs All 
# Dataset - extended (class 6 to 10) 
# Test - VisDrone (classwise)

# 1. Install Dependencies


In [1]:
!git clone https://github.com/mohitsharma-iitj/SEblock_yolov8.git
%cd SEblock_yolov8
!pip install -r requirements.txt


Cloning into 'SEblock_yolov8'...
remote: Enumerating objects: 205, done.
remote: Counting objects: 100% (205/205), done.
remote: Compressing objects: 100% (196/196), done.
remote: Total 205 (delta 36), reused 0 (delta 0), pack-reused 0
Receiving objects: 100% (205/205), 3.09 MiB | 14.26 MiB/s, done.
Resolving deltas: 100% (36/36), done.
/kaggle/working/SEblock_yolov8
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.2/41.2 kB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 73.3 MB/s eta 0:00:00:00:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.6/8.6 MB 83.3 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 530.8/530.8 kB 26.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.5/137.5 MB 12.0 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.7/87.7 MB 18.9 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 189.0/189.0 kB 11.4 MB/s eta

# 2. Create dataset function (Annotation)
as the format of annotation is as - object-class,x_center,y_center,width,height (all normalized as this is input format for YOLO5 prediction)

given annotation (in pixel) - bbox_left, bbox_top, bbox_width, bbox_height, score, object_category, truncation, occlusion

In [8]:
import os
import shutil
from sklearn.model_selection import train_test_split
from PIL import Image



os.makedirs('/kaggle/working/yolo_annotations', exist_ok=True)
os.makedirs( '/kaggle/working/yolo_dataset', exist_ok=True)

# Function to get image dimensions
def get_image_dimensions(image_path):
    with Image.open(image_path) as img:
        return img.width, img.height

# Function to convert annotations
def convert_to_yolo_format(filename, annotations, img_width, img_height, given_ ,recieved_path):
    output_annotations_path = recieved_path
    with open(os.path.join(output_annotations_path, filename), 'w') as out_file:
        for annotation in annotations:
            if len(annotation) != 8:
                print(f"Error parsing annotation in {filename}: {annotation}")
                continue
            x_min, y_min, width, height, score, class_id, truncation, occlusion = map(float, annotation)
            if(class_id==0):
                continue
            # Convert to YOLO format
            class_id = class_id -1
            if(class_id == given_):
                class_id = 0
                center_x = x_min + (width / 2.0)
                center_y = y_min + (height / 2.0)
                center_x /= img_width
                center_y /= img_height
                width /= img_width
                height /= img_height

                out_file.write(f"{int(class_id)} {center_x} {center_y} {width} {height}\n")


In [9]:
def create_files_train_val(given_):
    # Process all annotation files    # Paths to the dataset
    annotations_path = '/kaggle/input/extended-dataset/Expanded_dataset/annotations'
    images_path = '/kaggle/input/extended-dataset/Expanded_dataset/images'
    output_annotations_path = '/kaggle/working/yolo_annotations'
    
    shutil.rmtree('/kaggle/working/yolo_dataset')
    shutil.rmtree('/kaggle/working/yolo_annotations')

    os.makedirs('/kaggle/working/yolo_annotations', exist_ok=True)
    os.makedirs( os.path.join('/kaggle/working/yolo_dataset/images','train'), exist_ok=True)
    os.makedirs( os.path.join('/kaggle/working/yolo_dataset/images','val'), exist_ok=True)
    os.makedirs( os.path.join('/kaggle/working/yolo_dataset/labels','train'), exist_ok=True)
    os.makedirs( os.path.join('/kaggle/working/yolo_dataset/labels','val'), exist_ok=True)
    
    for annotation_file in os.listdir(annotations_path):
        image_file = annotation_file.replace('.txt', '.jpg')
        image_path = os.path.join(images_path, image_file)

        if not os.path.exists(image_path):
            continue

        img_width, img_height = get_image_dimensions(image_path)

        with open(os.path.join(annotations_path, annotation_file), 'r') as file:
            lines = file.readlines()
            annotations = [line.strip().split(',') for line in lines]
            convert_to_yolo_format(annotation_file, annotations, img_width, img_height, given_,output_annotations_path)


    # Split the data
    image_files = os.listdir(images_path)
    train_files, val_files = train_test_split(image_files, test_size=0.2, random_state=42)

    # Move files
    for file in train_files:
        shutil.copy(os.path.join(images_path, file), '/kaggle/working/yolo_dataset/images/train/')
        shutil.copy(os.path.join(output_annotations_path, file.replace('.jpg', '.txt')), '/kaggle/working/yolo_dataset/labels/train/')

    for file in val_files:
        shutil.copy(os.path.join(images_path, file), '/kaggle/working/yolo_dataset/images/val/')
        shutil.copy(os.path.join(output_annotations_path, file.replace('.jpg', '.txt')), '/kaggle/working/yolo_dataset/labels/val/')


## for test from visdrone

In [10]:
def create_files_test(given_):
    # Paths to the dataset
    annotations_path = '/kaggle/input/visdrone-dataset/VisDrone2019-DET-test-dev/VisDrone2019-DET-test-dev/annotations'
    images_path = '/kaggle/input/visdrone-dataset/VisDrone2019-DET-test-dev/VisDrone2019-DET-test-dev/images'
        
    output_dataset_path = '/kaggle/working/yolo_dataset'
    output_annotations_path = os.path.join(output_dataset_path, 'labels/test')
    output_images_path = os.path.join(output_dataset_path, 'images/test')

    os.makedirs(output_annotations_path, exist_ok=True)
    os.makedirs(output_images_path, exist_ok=True)

    
    # Process all annotation files
    for annotation_file in os.listdir(annotations_path):
        image_file = annotation_file.replace('.txt', '.jpg')
        image_path = os.path.join(images_path, image_file)

        if not os.path.exists(image_path):
            continue

        img_width, img_height = get_image_dimensions(image_path)

        with open(os.path.join(annotations_path, annotation_file), 'r') as file:
            lines = file.readlines()
            annotations = [line.strip().split(',') for line in lines]
            convert_to_yolo_format(annotation_file, annotations, img_width, img_height, given_,output_annotations_path)

            # Copy the image to the new dataset location
            output_image_path = os.path.join(output_images_path, image_file)
            if not os.path.exists(output_image_path):
                Image.open(image_path).save(output_image_path)

In [11]:
def classes_label(class_name):
    create_files_train_val(class_name)
    create_files_test(class_name)


# 3. Import yolov8

In [12]:
import wandb
from ultralytics import YOLO


# 4. Train Class wise



# Class 6 - tricycle

In [13]:
classes_label(6)


data_config = """
train: /kaggle/working/yolo_dataset/images/train
val: /kaggle/working/yolo_dataset/images/val
test: /kaggle/working/yolo_dataset/images/test

nc: 1  # number of classes 
names: ['tricycle']  # class names
"""

with open('/kaggle/working/yolo_dataset/data6.yaml', 'w') as file:
    file.write(data_config)


In [14]:
!export WANDB_MODE=disabled
wandb.init(mode="disabled")
    
model = YOLO('/kaggle/working/SEblock_yolov8/ultralytics/cfg/models/v8/yolov8_SEModule.yaml').load('/kaggle/working/SEblock_yolov8/yolov8n.pt')  
    
# Train the model
results = model.train(data='/kaggle/working/yolo_dataset/data6.yaml', epochs=25, imgsz=640, batch=16, name='tricycle_1')




WARNING ⚠️ no model scale passed. Assuming scale='n'.

                   from  n    params  module                                       arguments                     
  0                  -1  1       464  ultralytics.nn.modules.conv.Conv             [3, 16, 3, 2]                 
  1                  -1  1      4672  ultralytics.nn.modules.conv.Conv             [16, 32, 3, 2]                
  2                  -1  1      7360  ultralytics.nn.modules.block.C2f             [32, 32, 1, True]             
  3                  -1  1     18560  ultralytics.nn.modules.conv.Conv             [32, 64, 3, 2]                
  4                  -1  2     49664  ultralytics.nn.modules.block.C2f             [64, 64, 2, True]             
  5                  -1  1     73984  ultralytics.nn.modules.conv.Conv             [64, 128, 3, 2]               
  6                  -1  2    197632  ultralytics.nn.modules.block.C2f             [128, 128, 2, True]           
  7                  -1  1    295

In [15]:
val_results = model.val(data='/kaggle/working/yolo_dataset/data6.yaml', split='val')




Ultralytics YOLOv8.0.147 🚀 Python-3.10.13 torch-2.1.2 CUDA:0 (Tesla P100-PCIE-16GB, 16269MiB)
YOLOv8_SEModule summary (fused): 188 layers, 3007575 parameters, 0 gradients, 8.1 GFLOPs
val: Scanning /kaggle/working/yolo_dataset/labels/val.cache... 261 images, 233 backgrounds, 0 corrupt: 100%|██████████| 261/261 [00:00<?, ?it/s]
/opt/conda/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 17/17 [00:05<00:00,  3.35it/s]
                   all        261         51   0.000455      0.647     0.0405     0.0187
Speed: 0.1ms preprocess, 6.9ms inference, 0.0ms loss, 2.1ms postprocess per image
Saving /kaggle/working/SEblock_yolov8/runs/detect/tricycle_1/predictions.json...
Results saved to /kaggle/working/SEblock_yol

In [16]:
# Load the best model
model = YOLO('/kaggle/working/SEblock_yolov8/runs/detect/tricycle_1/weights/best.pt')


# Evaluate the model on the new dataset
test_results = model.val(data='/kaggle/working/yolo_dataset/data6.yaml', split='test')

Ultralytics YOLOv8.0.147 🚀 Python-3.10.13 torch-2.1.2 CUDA:0 (Tesla P100-PCIE-16GB, 16269MiB)
YOLOv8_SEModule summary (fused): 188 layers, 3007575 parameters, 0 gradients, 8.1 GFLOPs
val: Scanning /kaggle/working/yolo_dataset/labels/test... 1610 images, 1365 backgrounds, 0 corrupt: 100%|██████████| 1610/1610 [00:00<00:00, 1912.69it/s]
val: New cache created: /kaggle/working/yolo_dataset/labels/test.cache
/opt/conda/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 101/101 [00:16<00:00,  6.20it/s]
                   all       1610        530   0.000168      0.142    0.00113   0.000138
Speed: 0.1ms preprocess, 1.5ms inference, 0.0ms loss, 2.1ms postprocess per image
Saving /kaggle/working/SEblock_yolov8/runs/

# Class 7 - awning-tricycle

In [17]:
classes_label(7)


data_config = """
train: /kaggle/working/yolo_dataset/images/train
val: /kaggle/working/yolo_dataset/images/val
test: /kaggle/working/yolo_dataset/images/test

nc: 1  # number of classes 
names: ['awning-tricycle']  # class names
"""

with open('/kaggle/working/yolo_dataset/data7.yaml', 'w') as file:
    file.write(data_config)




In [18]:
!export WANDB_MODE=disabled
wandb.init(mode="disabled")
 
    
model = YOLO('/kaggle/working/SEblock_yolov8/ultralytics/cfg/models/v8/yolov8_SEModule.yaml').load('/kaggle/working/SEblock_yolov8/yolov8n.pt')  

# Train the model
results = model.train(data='/kaggle/working/yolo_dataset/data7.yaml', epochs=25, imgsz=640, batch=16, name='awning-tricycle_1')



/opt/conda/lib/python3.10/pty.py:89: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid, fd = os.forkpty()
WARNING ⚠️ no model scale passed. Assuming scale='n'.

                   from  n    params  module                                       arguments                     
  0                  -1  1       464  ultralytics.nn.modules.conv.Conv             [3, 16, 3, 2]                 
  1                  -1  1      4672  ultralytics.nn.modules.conv.Conv             [16, 32, 3, 2]                
  2                  -1  1      7360  ultralytics.nn.modules.block.C2f             [32, 32, 1, True]             
  3                  -1  1     18560  ultralytics.nn.modules.conv.Conv             [32, 64, 3, 2]                
  4                  -1  2     49664  ultralytics.nn.modules.block.C2f             [64, 64, 2, True]             
  5                  -1  1     73984  ultral

In [19]:
val_results = model.val(data='/kaggle/working/yolo_dataset/data7.yaml', split='val')


Ultralytics YOLOv8.0.147 🚀 Python-3.10.13 torch-2.1.2 CUDA:0 (Tesla P100-PCIE-16GB, 16269MiB)
YOLOv8_SEModule summary (fused): 188 layers, 3007575 parameters, 0 gradients, 8.1 GFLOPs
val: Scanning /kaggle/working/yolo_dataset/labels/val.cache... 261 images, 232 backgrounds, 0 corrupt: 100%|██████████| 261/261 [00:00<?, ?it/s]
/opt/conda/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 17/17 [00:03<00:00,  5.64it/s]
                   all        261         84          0          0          0          0
Speed: 0.2ms preprocess, 6.9ms inference, 0.0ms loss, 0.4ms postprocess per image
Results saved to /kaggle/working/SEblock_yolov8/runs/detect/awning-tricycle_1


In [20]:
# Load the best model
model = YOLO('/kaggle/working/SEblock_yolov8/runs/detect/awning-tricycle_1/weights/best.pt')


# Evaluate the model on the new dataset
test_results = model.val(data='/kaggle/working/yolo_dataset/data7.yaml', split='test')

Ultralytics YOLOv8.0.147 🚀 Python-3.10.13 torch-2.1.2 CUDA:0 (Tesla P100-PCIE-16GB, 16269MiB)
YOLOv8_SEModule summary (fused): 188 layers, 3007575 parameters, 0 gradients, 8.1 GFLOPs
val: Scanning /kaggle/working/yolo_dataset/labels/test... 1610 images, 1377 backgrounds, 0 corrupt: 100%|██████████| 1610/1610 [00:00<00:00, 1812.11it/s]
val: New cache created: /kaggle/working/yolo_dataset/labels/test.cache
/opt/conda/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 101/101 [00:12<00:00,  7.79it/s]
                   all       1610        599          0          0          0          0
Speed: 0.1ms preprocess, 1.8ms inference, 0.0ms loss, 0.4ms postprocess per image
Results saved to /kaggle/working/SEblock_yo

# Class 8 - bus

In [21]:
classes_label(8)


data_config = """
train: /kaggle/working/yolo_dataset/images/train
val: /kaggle/working/yolo_dataset/images/val
test: /kaggle/working/yolo_dataset/images/test

nc: 1  # number of classes 
names: ['bus']  # class names
"""

with open('/kaggle/working/yolo_dataset/data8.yaml', 'w') as file:
    file.write(data_config)




In [22]:
!export WANDB_MODE=disabled
wandb.init(mode="disabled")
    
    
model = YOLO('/kaggle/working/SEblock_yolov8/ultralytics/cfg/models/v8/yolov8_SEModule.yaml').load('/kaggle/working/SEblock_yolov8/yolov8n.pt')  

# Train the model
results = model.train(data='/kaggle/working/yolo_dataset/data8.yaml', epochs=25, imgsz=640, batch=16, name='bus_1')


/opt/conda/lib/python3.10/pty.py:89: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid, fd = os.forkpty()
WARNING ⚠️ no model scale passed. Assuming scale='n'.

                   from  n    params  module                                       arguments                     
  0                  -1  1       464  ultralytics.nn.modules.conv.Conv             [3, 16, 3, 2]                 
  1                  -1  1      4672  ultralytics.nn.modules.conv.Conv             [16, 32, 3, 2]                
  2                  -1  1      7360  ultralytics.nn.modules.block.C2f             [32, 32, 1, True]             
  3                  -1  1     18560  ultralytics.nn.modules.conv.Conv             [32, 64, 3, 2]                
  4                  -1  2     49664  ultralytics.nn.modules.block.C2f             [64, 64, 2, True]             
  5                  -1  1     73984  ultral

In [23]:
val_results = model.val(data='/kaggle/working/yolo_dataset/data8.yaml', split='val')

Ultralytics YOLOv8.0.147 🚀 Python-3.10.13 torch-2.1.2 CUDA:0 (Tesla P100-PCIE-16GB, 16269MiB)
YOLOv8_SEModule summary (fused): 188 layers, 3007575 parameters, 0 gradients, 8.1 GFLOPs
val: Scanning /kaggle/working/yolo_dataset/labels/val.cache... 261 images, 135 backgrounds, 0 corrupt: 100%|██████████| 261/261 [00:00<?, ?it/s]
/opt/conda/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 17/17 [00:04<00:00,  3.42it/s]
                   all        261        336   0.000128     0.0298    6.6e-05    1.5e-05
Speed: 0.1ms preprocess, 1.8ms inference, 0.0ms loss, 5.0ms postprocess per image
Saving /kaggle/working/SEblock_yolov8/runs/detect/bus_1/predictions.json...
Results saved to /kaggle/working/SEblock_yolov8/r

In [24]:
# Load the best model
model = YOLO('/kaggle/working/SEblock_yolov8/runs/detect/bus_1/weights/best.pt')


# Evaluate the model on the new dataset
test_results = model.val(data='/kaggle/working/yolo_dataset/data8.yaml', split='test')

Ultralytics YOLOv8.0.147 🚀 Python-3.10.13 torch-2.1.2 CUDA:0 (Tesla P100-PCIE-16GB, 16269MiB)
YOLOv8_SEModule summary (fused): 188 layers, 3007575 parameters, 0 gradients, 8.1 GFLOPs
val: Scanning /kaggle/working/yolo_dataset/labels/test... 1610 images, 772 backgrounds, 0 corrupt: 100%|██████████| 1610/1610 [00:01<00:00, 1555.88it/s]
val: New cache created: /kaggle/working/yolo_dataset/labels/test.cache
/opt/conda/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 101/101 [00:18<00:00,  5.36it/s]
                   all       1610       2940   0.000147     0.0241   7.53e-05   1.98e-05
Speed: 0.1ms preprocess, 1.4ms inference, 0.0ms loss, 3.1ms postprocess per image
Saving /kaggle/working/SEblock_yolov8/runs/d

# Class 9 - motor

In [25]:
classes_label(9)


data_config = """
train: /kaggle/working/yolo_dataset/images/train
val: /kaggle/working/yolo_dataset/images/val
test: /kaggle/working/yolo_dataset/images/test

nc: 1  # number of classes 
names: ['motor']  # class names
"""

with open('/kaggle/working/yolo_dataset/data9.yaml', 'w') as file:
    file.write(data_config)




In [26]:
!export WANDB_MODE=disabled
wandb.init(mode="disabled")
    
    
model = YOLO('/kaggle/working/SEblock_yolov8/ultralytics/cfg/models/v8/yolov8_SEModule.yaml').load('/kaggle/working/SEblock_yolov8/yolov8n.pt')  

# Train the model
results = model.train(data='/kaggle/working/yolo_dataset/data9.yaml', epochs=25, imgsz=640, batch=16, name='motor_1')




/opt/conda/lib/python3.10/pty.py:89: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid, fd = os.forkpty()
WARNING ⚠️ no model scale passed. Assuming scale='n'.

                   from  n    params  module                                       arguments                     
  0                  -1  1       464  ultralytics.nn.modules.conv.Conv             [3, 16, 3, 2]                 
  1                  -1  1      4672  ultralytics.nn.modules.conv.Conv             [16, 32, 3, 2]                
  2                  -1  1      7360  ultralytics.nn.modules.block.C2f             [32, 32, 1, True]             
  3                  -1  1     18560  ultralytics.nn.modules.conv.Conv             [32, 64, 3, 2]                
  4                  -1  2     49664  ultralytics.nn.modules.block.C2f             [64, 64, 2, True]             
  5                  -1  1     73984  ultral

In [27]:

val_results = model.val(data='/kaggle/working/yolo_dataset/data9.yaml', split='val')


Ultralytics YOLOv8.0.147 🚀 Python-3.10.13 torch-2.1.2 CUDA:0 (Tesla P100-PCIE-16GB, 16269MiB)
YOLOv8_SEModule summary (fused): 188 layers, 3007575 parameters, 0 gradients, 8.1 GFLOPs
val: Scanning /kaggle/working/yolo_dataset/labels/val.cache... 261 images, 146 backgrounds, 0 corrupt: 100%|██████████| 261/261 [00:00<?, ?it/s]
/opt/conda/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 17/17 [02:48<00:00,  9.88s/it]
                   all        261        790          0          0          0          0
Speed: 0.1ms preprocess, 1.7ms inference, 0.0ms loss, 3.2ms postprocess per image
Saving /kaggle/working/SEblock_yolov8/runs/detect/motor_1/predictions.json...
Results saved to /kaggle/working/SEblock_yolov8

In [28]:
# Load the best model
model = YOLO('/kaggle/working/SEblock_yolov8/runs/detect/motor_1/weights/best.pt')


# Evaluate the model on the new dataset
test_results = model.val(data='/kaggle/working/yolo_dataset/data9.yaml', split='test')

Ultralytics YOLOv8.0.147 🚀 Python-3.10.13 torch-2.1.2 CUDA:0 (Tesla P100-PCIE-16GB, 16269MiB)
YOLOv8_SEModule summary (fused): 188 layers, 3007575 parameters, 0 gradients, 8.1 GFLOPs
val: Scanning /kaggle/working/yolo_dataset/labels/test... 1610 images, 816 backgrounds, 0 corrupt: 100%|██████████| 1610/1610 [00:03<00:00, 531.68it/s]
val: New cache created: /kaggle/working/yolo_dataset/labels/test.cache
/opt/conda/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 101/101 [00:19<00:00,  5.18it/s]
                   all       1610       5845          0          0          0          0
Speed: 0.1ms preprocess, 1.5ms inference, 0.0ms loss, 3.2ms postprocess per image
Saving /kaggle/working/SEblock_yolov8/runs/de

# Class 10- others

In [29]:
classes_label(10)


data_config = """
train: /kaggle/working/yolo_dataset/images/train
val: /kaggle/working/yolo_dataset/images/val
test: /kaggle/working/yolo_dataset/images/test

nc: 1  # number of classes 
names: ['others']  # class names
"""

with open('/kaggle/working/yolo_dataset/data10.yaml', 'w') as file:
    file.write(data_config)


In [30]:
!export WANDB_MODE=disabled
wandb.init(mode="disabled")
    
model = YOLO('/kaggle/working/SEblock_yolov8/ultralytics/cfg/models/v8/yolov8_SEModule.yaml').load('/kaggle/working/SEblock_yolov8/yolov8n.pt')  

# Train the model
results = model.train(data='/kaggle/working/yolo_dataset/data10.yaml', epochs=25, imgsz=640, batch=16, name='others_1')



/opt/conda/lib/python3.10/pty.py:89: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid, fd = os.forkpty()
WARNING ⚠️ no model scale passed. Assuming scale='n'.

                   from  n    params  module                                       arguments                     
  0                  -1  1       464  ultralytics.nn.modules.conv.Conv             [3, 16, 3, 2]                 
  1                  -1  1      4672  ultralytics.nn.modules.conv.Conv             [16, 32, 3, 2]                
  2                  -1  1      7360  ultralytics.nn.modules.block.C2f             [32, 32, 1, True]             
  3                  -1  1     18560  ultralytics.nn.modules.conv.Conv             [32, 64, 3, 2]                
  4                  -1  2     49664  ultralytics.nn.modules.block.C2f             [64, 64, 2, True]             
  5                  -1  1     73984  ultral

In [31]:
val_results = model.val(data='/kaggle/working/yolo_dataset/data10.yaml', split='val')

Ultralytics YOLOv8.0.147 🚀 Python-3.10.13 torch-2.1.2 CUDA:0 (Tesla P100-PCIE-16GB, 16269MiB)
YOLOv8_SEModule summary (fused): 188 layers, 3007575 parameters, 0 gradients, 8.1 GFLOPs
val: Scanning /kaggle/working/yolo_dataset/labels/val.cache... 261 images, 239 backgrounds, 0 corrupt: 100%|██████████| 261/261 [00:00<?, ?it/s]
/opt/conda/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 17/17 [01:26<00:00,  5.11s/it]
                   all        261         28          0          0          0          0
Speed: 0.1ms preprocess, 1.8ms inference, 0.0ms loss, 3.4ms postprocess per image
Saving /kaggle/working/SEblock_yolov8/runs/detect/others_1/predictions.json...
Results saved to /kaggle/working/SEblock_yolov

In [32]:
# Load the best model
model = YOLO('/kaggle/working/SEblock_yolov8/runs/detect/others_1/weights/best.pt')


# Evaluate the model on the new dataset
test_results = model.val(data='/kaggle/working/yolo_dataset/data10.yaml', split='test')

Ultralytics YOLOv8.0.147 🚀 Python-3.10.13 torch-2.1.2 CUDA:0 (Tesla P100-PCIE-16GB, 16269MiB)
YOLOv8_SEModule summary (fused): 188 layers, 3007575 parameters, 0 gradients, 8.1 GFLOPs
val: Scanning /kaggle/working/yolo_dataset/labels/test... 1610 images, 1464 backgrounds, 0 corrupt: 100%|██████████| 1610/1610 [00:00<00:00, 1780.61it/s]
val: New cache created: /kaggle/working/yolo_dataset/labels/test.cache
/opt/conda/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 101/101 [00:17<00:00,  5.68it/s]
                   all       1610        265   4.17e-06    0.00755    2.1e-06   5.24e-07
Speed: 0.1ms preprocess, 1.6ms inference, 0.0ms loss, 3.2ms postprocess per image
Saving /kaggle/working/SEblock_yolov8/runs/